In [4]:
import pandas as pd
import os
import gc

# --- Paths for CTU-13 dataset ---
dataset_files = {
    '1': '1/capture20110810.binetflow',
    '2': '2/capture20110811.binetflow',
    '5': '5/capture20110815-2.binetflow',
    '9': '9/capture20110817.binetflow',
    '13': '13/capture20110815-3.binetflow'
}
# --- End paths ---

# Tentukan nama folder untuk menyimpan hasil sampel
output_folder = 'sampled_data'

# Buat dictionary kosong untuk menampung semua DataFrame LENGKAP
# Variabel ini akan 'hidup' setelah sel ini dijalankan
dict_of_dataframes = {}

In [5]:
print("Memulai proses: Membaca data & mengambil sampel...")

# 1. Buat folder output jika belum ada
try:
    os.makedirs(output_folder, exist_ok=True)
    print(f"Folder output '{output_folder}' telah disiapkan.")
except Exception as e:
    print(f"Gagal membuat folder '{output_folder}': {e}")


# 2. Loop, baca, simpan ke dict, ambil sampel, dan simpan sampel
for key, filepath in dataset_files.items():
    try:
        print(f"\n--- Memproses Dataset {key} ---")
        print(f"Membaca file: {filepath}...")
        
        # a. Baca file
        df = pd.read_csv(filepath, low_memory=False)
        total_rows = len(df)
        
        # b. Simpan DataFrame LENGKAP ke dictionary
        # Ini memenuhi permintaan Anda untuk mengaksesnya nanti
        dict_of_dataframes[key] = df
        print(f"Dataset {key} ({total_rows} baris) telah dimuat ke memori.")

        # c. Tentukan jumlah sampel
        sample_size = min(100, total_rows)
        if sample_size < 100:
            print(f"Peringatan: File {key} hanya memiliki {total_rows} baris. Mengambil {sample_size} baris.")
        else:
            print(f"Mengambil 100 baris acak dari file {key}...")

        # d. Ambil sampel dari DataFrame yang baru saja dibaca
        sampled_df_botnet_not_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=25, random_state=42)

        sampled_df_botnet_and_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=25, random_state=42)

        sampled_df_no_botnet_no_spam = df[
            ~df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=50, random_state=42)

        # Gabungkan ketiga DataFrame menjadi satu
        sampled_df = pd.concat([sampled_df_botnet_not_spam, sampled_df_botnet_and_spam, sampled_df_no_botnet_no_spam], ignore_index=True)

        # Acak urutan baris
        sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Hapus ketiga variabel yang sudah digabungkan agar tidak memenuhi memori
        del sampled_df_botnet_not_spam
        del sampled_df_botnet_and_spam
        del sampled_df_no_botnet_no_spam

        # Memastikan variabel-variabel sudah dihapus dan memori lebih efisien
        gc.collect()

        # e. Tentukan nama file output dan simpan sampel
        original_folder = os.path.dirname(filepath)  # Dapatkan folder asli dari path file
        folder_name = os.path.basename(original_folder)  # Ambil nama folder seperti '1', '2', dll.
        
        # Buat folder di dalam sampled_data dengan nama folder asal
        folder_output_path = os.path.join(output_folder, folder_name)
        os.makedirs(folder_output_path, exist_ok=True)

        base_filename = os.path.basename(filepath)  # Ambil nama file asli
        output_filepath = os.path.join(folder_output_path, base_filename)  # Gabungkan dengan folder output
        
        sampled_df.to_csv(output_filepath, index=False)
        print(f"Sukses! Sampel disimpan ke '{output_filepath}'")
        
    except FileNotFoundError:
        print(f"PERINGATAN: File tidak ditemukan di '{filepath}'. Dataset {key} akan dilewati.")
    except Exception as e:
        print(f"Gagal memproses file {filepath}: {e}. Dataset {key} akan dilewati.")

print("\n--- Semua proses selesai ---")
print(f"Berhasil memuat {len(dict_of_dataframes)} dataset ke memori.")

Memulai proses: Membaca data & mengambil sampel...
Folder output 'sampled_data' telah disiapkan.

--- Memproses Dataset 1 ---
Membaca file: 1/capture20110810.binetflow...
Dataset 1 (2824636 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 1...
Sukses! Sampel disimpan ke 'sampled_data\1\capture20110810.binetflow'

--- Memproses Dataset 2 ---
Membaca file: 2/capture20110811.binetflow...
Dataset 2 (1808122 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 2...
Sukses! Sampel disimpan ke 'sampled_data\2\capture20110811.binetflow'

--- Memproses Dataset 5 ---
Membaca file: 5/capture20110815-2.binetflow...
Dataset 5 (129832 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 5...
Sukses! Sampel disimpan ke 'sampled_data\5\capture20110815-2.binetflow'

--- Memproses Dataset 9 ---
Membaca file: 9/capture20110817.binetflow...
Dataset 9 (2087508 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 9...
Sukses! Sampel disimpan ke 'sample

In [6]:
# Cek key (dataset) apa saja yang tersedia di dictionary
print(f"Dataset yang berhasil dimuat ke memori: {list(dict_of_dataframes.keys())}")

# Contoh: Mengakses dan menampilkan 5 baris pertama dari dataset '5' (yang LENGKAP)
try:
    print("\nMenampilkan .head() dari dataset '5' (diambil dari memori):")
    
    # Ini adalah cara Anda mengakses data LENGKAP-nya
    df_5_full = dict_of_dataframes['5']
    
    display(df_5_full.head())
    print(f"Ukuran dataset '5' di memori: {df_5_full.shape}")
    
except KeyError:
    print("Dataset '5' tidak berhasil dimuat (mungkin error atau tidak ditemukan).")

Dataset yang berhasil dimuat ke memori: ['1', '2', '5', '9', '13']

Menampilkan .head() dari dataset '5' (diambil dari memori):


,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/15 16:43:28.078942,0.000000,tcp,114.33.245.44,6881,?>,147.32.84.118,1567,RA_,0.0,0.0,1,60,60,flow=Background
1,2011/08/15 16:43:32.283576,13.431962,tcp,212.93.105.52,49237,->,147.32.84.229,80,SRA_SA,0.0,0.0,6,388,208,flow=Background-TCP-Established
2,2011/08/15 16:43:32.456441,13.350228,tcp,212.93.105.52,14906,->,147.32.84.229,13363,SRA_SA,0.0,0.0,6,388,208,flow=Background-TCP-Established
3,2011/08/15 16:43:32.850648,13.010090,tcp,212.93.105.52,60349,->,147.32.84.229,443,SRA_SA,0.0,0.0,6,388,208,flow=Background-TCP-Established
4,2011/08/15 16:45:09.305002,20.990047,tcp,115.127.24.116,3198,->,147.32.84.229,443,SR_SA,0.0,0.0,5,308,122,flow=Background-TCP-Established


Ukuran dataset '5' di memori: (129832, 15)
